In [1]:
"""
This programm is not essential. 

It just allows you to see your dataset with boxes.
Don't forget to %pip install scikit-image
"""

"\nThis programm is not essential. \n\nIt just allows you to see your dataset with boxes.\nDon't forget to %pip install scikit-image\n"

In [2]:
%matplotlib inline
import random
import time
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import tensorflow as tf
from pylab import rcParams
import os
import json



MODEL_PATH = '../data/model_artif/'
SAMPLES_PATHS = ['Images+json/']



In [3]:
def LoadImage(fname):
    return io.imread(fname)[:,:] / 255.


In [4]:
def LoadAnnotation(fname):
    left=[]
    top=[]
    right=[]
    bottom=[]
    
    with open(fname) as data_file:    
        data = json.load(data_file)       
    
    height, width = data["size"]["height"],  data["size"]["width"]
          
    
    for bbox in data["objects"]:
        
        #We need to rectify flip_vert and flip_hori
        if "flip_vert" in data["tags"]:
            bbox["points"]["exterior"][0][0]=width/2-(bbox["points"]["exterior"][0][0]-width/2)
            bbox["points"]["exterior"][1][0]=width/2-(bbox["points"]["exterior"][1][0]-width/2)
        if "flip_hori" in data["tags"]:
            bbox["points"]["exterior"][0][1]=height/2-(bbox["points"]["exterior"][0][1]-height/2)
            bbox["points"]["exterior"][1][1]=height/2-(bbox["points"]["exterior"][1][1]-height/2)
                
                
        if (bbox["points"]["exterior"][0][0]<bbox["points"]["exterior"][1][0]):
            left.append(bbox["points"]["exterior"][0][0])
            right.append(bbox["points"]["exterior"][1][0])
        else :
            left.append(bbox["points"]["exterior"][1][0])
            right.append(bbox["points"]["exterior"][0][0])
        if (bbox["points"]["exterior"][0][1]<bbox["points"]["exterior"][1][1]):
            top.append(bbox["points"]["exterior"][0][1])
            bottom.append(bbox["points"]["exterior"][1][1])
        else :
            top.append(bbox["points"]["exterior"][1][1])
            bottom.append(bbox["points"]["exterior"][0][1])
    
    return [left, top, right, bottom]

In [5]:
def ReadDirFiles(dname):
    paths = []
    for file in os.listdir(os.path.join(dname, "img")):
        if not(file.startswith(".ipynb")):
            bname = os.path.basename(file).split(".")[0]

            img_name = os.path.join(dname, "img", file)
            ann_name = os.path.join(dname, "ann", bname + ".json")
            paths.append((img_name, ann_name))
    return paths

In [6]:
def ReadPaths(paths):
    all_paths = []
    for path in paths:
        temp_paths = ReadDirFiles(path)
        all_paths.extend(temp_paths)
    return all_paths

In [7]:
def get_tags(fname):
    with open(fname) as data_file:
        data = json.load(data_file)
    tags = data["tags"]
    return tags

In [8]:
# Split test data: 80% train, 10% validation, 10% test

def train_test_split(paths, train_rate=0.8):
    train_paths = []
    valid_paths=[]
    test_paths = []
    
    indices = np.arange(len(paths))
    np.random.shuffle(indices)
    
    nTrain = int(len(paths) * train_rate)
    nValid = (len(paths) - nTrain) // 2
    nTest = len(paths) - nTrain - nValid
    count=0
    
    for path in paths:
        if (indices[count]<nTrain):
            train_paths.append(path)
        else :
            if (indices[count]<nValid+nTrain):
                valid_paths.append(path)
            else:
                test_paths.append(path)
        count+=1
    
    return train_paths, valid_paths, test_paths

In [9]:
def LoadData(paths):
    xs = []
    ys = []
    for ex_paths in paths:
        img_path = ex_paths[0]
        ann_path = ex_paths[1]
        xs.append(LoadImage(img_path))
        #if (img_path.startswith('Images+json/img/monobloc')):
        ys.append(LoadAnnotation(ann_path))
        #else :
        #    ys.append()
    
    return np.array(xs), np.array(ys)


In [10]:
all_paths = ReadPaths(SAMPLES_PATHS)
tr_paths, va_paths, te_paths = train_test_split(all_paths,0.9)

print(len(tr_paths))
print(len(va_paths))
print(len(te_paths))



X_train, Y_train = LoadData(tr_paths)
X_valid, Y_valid = LoadData(va_paths)
X_test, Y_test = LoadData(te_paths)

print("check shapes:")
print("X_train - ", X_train.shape)
print("Y_train - ", Y_train.shape)
print("X_valid - ", X_valid.shape)
print("Y_valid - ", Y_valid.shape)
print("X_test - ", X_test.shape)
print("Y_test - ", Y_test.shape)



7639
424
425


FileNotFoundError: [Errno 2] No such file or directory: 'Images+json/ann/ztj.json'

In [ ]:
#Show sample images


def show_image(image, labels):
    for i in range (len(labels[0])) :
        rect = Rectangle((labels[0][i], labels[1][i]), labels[2][i]-labels[0][i], labels[3][i]-labels[1][i], edgecolor='r', fill=False)
        plt.imshow(image)
        gca = plt.gca()
        gca.add_patch(rect)
    
def plot_images(images, labels):
    rcParams['figure.figsize'] = 14, 8
    plt.gray()
    fig = plt.figure()
    for i in range(min(9, images.shape[0])):
        fig.add_subplot(3, 3, i+1)
        show_image(images[i], labels[i])
    plt.show()    
    
xs = [random.randint(0, Y_train.shape[0]-1) for _ in range(98)]
plot_images(X_train[xs], Y_train[xs])




In [ ]:
#Show the monobloc you want
imono,lmono = LoadData([["Images+json/img/monobloc (0)_qMf7EgNi4r.png","Images+json/ann/monobloc (0)_qMf7EgNi4r.json"],["Images+json/img/monobloc (0)_SQVhqefiar.png","Images+json/ann/monobloc (0)_SQVhqefiar.json"]])

plot_images(imono,lmono)
print (lmono)